In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

#CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag', '0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(type(incremental_flag))

<class 'str'>


#CREATING DIMENSION MODEL

###Fetch Relative Columns

In [0]:
df_src = spark.sql('''
    select DISTINCT(Model_ID) as Model_ID, Model_Category 
    from parquet.`abfss://silver@carannweshadatalake.dfs.core.windows.net/carsales`''')

In [0]:
display(df_src)

Model_ID,Model_Category
Mah-M167,Mah
Che-M47,Che
Toy-M205,Toy
BMW-M249,BMW
Mer-M122,Mer
Hon-M215,Hon
Nis-M82,Nis
Toy-M206,Toy
Mar-M139,Mar
Ren-M207,Ren


###dim_model Sink - Initial & Incremental

In [0]:
if not spark.catalog.tableExists('cars_catalog.gold.dim_model'):

    df_sink = spark.sql('''
                        select 1 as dim_model_key, Model_ID, Model_Category
                        from parquet.`abfss://silver@carannweshadatalake.dfs.core.windows.net/carsales`
                        where 1=0
                        ''')
    
else:

    df_sink = spark.sql('''
                        select dim_model_key, Model_ID, Model_Category
                        from parquet.`abfss://silver@carannweshadatalake.dfs.core.windows.net/carsales`
                        ''')


{"ts": "2025-01-04 21:25:22,001", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `dim_model_key` cannot be resolved. Did you mean one of the following? [`Model_ID`, `Date_ID`, `Dealer_ID`, `Revenue`, `model_category`]. SQLSTATE: 42703", "context": {"errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o394.sql.\n: org.apache.spark.sql.catalyst.ExtendedAnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `dim_model_key` cannot be resolved. Did you mean one of the following? [`Model_ID`, `Date_ID`, `Dealer_ID`, `Revenue`, `model_category`]. SQLSTATE: 42703; line 2 pos 31;\n'Project ['dim_model_key, Model_ID#5112, Model_Category#5121]\n+- Relation [Branch_ID#5110,Dealer_ID#5111,Model_ID#5112,Revenue#5113L,Units_Sold#5114L,Date_ID#5115,Day#5116,Month#5117,Year

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2658164853829715>, line 11
      3     df_sink = spark.sql('''
      4                         select 1 as dim_model_key, Model_ID, Model_Category
      5                         from parquet.`abfss://silver@carannweshadatalake.dfs.core.windows.net/carsales`
      6                         where 1=0
      7                         ''')
      9 else:
---> 11     df_sink = spark.sql('''
     12                         select dim_model_key, Model_ID, Model_Category
     13                         from parquet.`abfss://silver@carannweshadatalake.dfs.core.windows.net/carsales`
     14                         ''')

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48  

###Filtering Old Records and New Records

In [0]:
df_filter = df_src.join(df_sink, df_src.Model_ID == df_sink.Model_ID, 'left').select(df_src.Model_ID, df_src.Model_Category, df_sink.dim_model_key)

In [0]:
display(df_filter)

Model_ID,Model_Category,dim_model_key
Mah-M167,Mah,null
Che-M47,Che,null
Toy-M205,Toy,null
BMW-M249,BMW,null
Mer-M122,Mer,null
Hon-M215,Hon,null
Nis-M82,Nis,null
Toy-M206,Toy,null
Mar-M139,Mar,null
Ren-M207,Ren,null


**df_filter_old**

In [0]:
df_filter_old = df_filter.filter(col('dim_model_key').isNotNull())


In [0]:
display(df_filter_old)

Model_ID,Model_Category,dim_model_key


**df_filter_new**

In [0]:
df_filter_new = df_filter.filter(col('dim_model_key').isNull()).select(df_src['Model_ID'], df_src['Model_Category'])

In [0]:
display(df_filter_new)

Model_ID,Model_Category
Mah-M167,Mah
Che-M47,Che
Toy-M205,Toy
BMW-M249,BMW
Mer-M122,Mer
Hon-M215,Hon
Nis-M82,Nis
Toy-M206,Toy
Mar-M139,Mar
Ren-M207,Ren


###Create Surrogate Key

**Fetch the Max Surrogate Key from Existing table**

In [0]:
if incremental_flag == '0':
    max_value=1
else:
    max_value_df = spark.sql('SELECT max(dim_model_key) FROM cars_catalog.gold.dim_model')
    max_value = max_value_df.collect()[0][0]

**Create the Surrogate key Column and ADD the Max Surrogate Key**

In [0]:
df_filter_new = df_filter_new.withColumn('dim_model_key', max_value + monotonically_increasing_id())

In [0]:
display(df_filter_new)

Model_ID,Model_Category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10


###Create Final DF - df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
display(df_final)

Model_ID,Model_Category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10


#SCD Type - 1 (UPSERT)

In [0]:
#Incremental Run
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    delta_tbl = DeltaTable.forPath(spark, 'abfss://gold@carannweshadatalake.dfs.core.windows.net/dim_model')

    delta_tbl.alias('trg').merge(df_final.alias('src'), 'trg.dim_model_key = src.dim_model_key')\
                          .whenMatchedUpdateAll()\
                          .whenNotMatchedInsertAll()\
                          .execute()

#Initial Run
else:
    df_final.write.format('delta')\
        .mode('overwrite')\
        .option('path','abfss://gold@carannweshadatalake.dfs.core.windows.net/dim_model')\
        .saveAsTable('cars_catalog.gold.dim_model')

In [0]:
%sql
select * from cars_catalog.gold.dim_model